In [63]:
from pyspark.sql import SparkSession 
from pyspark.sql.functions import col, udf, date_format, rank, desc, count, row_number, explode
from pyspark.sql.window import Window
from datetime import datetime
from typing import List, Tuple
from pyspark.sql.types import StringType, ArrayType
import emoji

In [ ]:
#python --version  3.8.13
#spark --version 3.4.1 Using Scala version 2.12.17
#emoji module --version 2.8.0 

###      ENFOQUE OPTIMIZACIÓN DE TIEMPO

#### Q1

In [64]:
def q1_time(file_path: str) -> List[Tuple[datetime.date, str]]:

    # Creamos la sesión de Spark y cargamos el archivo
    spark = SparkSession.builder.appName("TopDateUsers").getOrCreate()
    df = spark.read.json(file_path)

    # Convertimos la columna date (para que tome solo lo necesario)
    df = df.withColumn("date", date_format(col("date"), "yyyy-MM-dd").cast("date"))

    # Unimos los tweets originales y los tweets en quotedTweet
    df_combined = df.select("date", "user.username").union(df.select("quotedTweet.date", "quotedTweet.user.username").filter(col("quotedTweet.date").isNotNull() & col("quotedTweet.user.username").isNotNull()))

    #df con las 10 fechas con más tweets
    #esta parte se podría parametrizar, para este caso es valido ya que sabemos a priori el tamaño a mostrar en la lista,
    #pero de ser un tamaño n no sería conveniente.
    top_dates = df_combined.groupBy("date").count().orderBy(desc("count")).limit(10)

    
    #Obtenemos las 10 fechas con más tweets directamente como DataFrame
    top_dates_df = top_dates.select("date")

    #Convertimos las fechas en una lista de objetos datetime.date
    top_dates_list = [row.date for row in top_dates_df.collect()]

    # Limpiams el DataFrame intermedio
    top_dates_df.unpersist()
    
    # almacenamos en una lista las 10 fechas que más se repiten del df anterior
    #top_dates_list = top_dates.select("date").rdd.flatMap(lambda x: x).collect()


    #creamos un df que solo contiene las fechas que están en la lista que generamos
    filtered_df = df_combined.filter(col("date").isin(top_dates_list))

    #Limpiamos el DataFrame que agrupa los tweets (originales - retweets)
    df_combined.unpersist()
    
    # Agrupamos el DataFrame por fecha y usuario y calcula el recuento de cada grupo
    grouped_df = filtered_df.groupBy("date", "username").agg(count("*").alias("count"))

    # Usamos una ventana para obtener el usuario que más se repite por fecha mediante rank
    window_spec = Window.partitionBy("date").orderBy(desc("count"))
    top_users_df = grouped_df.withColumn("rank", row_number().over(window_spec)).filter(col("rank") == 1)

    #Seleccionar y ordenamos las columnas necesarias
    result_df = top_users_df.select("date", "username", "count").orderBy(desc("count"))

    # Recopilamos los resultados en una lista de tuplas
    result = result_df.collect()

    # Conviertimos el resultado en una lista de tuplas
    result_list = [(row.date, row.username) for row in result]
    

    return result_list

#### Q2

In [65]:
def q2_time(file_path: str) -> List[Tuple[str, int]]:
    # Creamos la sesión de Spark y cargamos el archivo
    spark = SparkSession.builder.appName("TopEmojis").getOrCreate()
    df = spark.read.json(file_path)

    # Función para dividir los emojis en caracteres individuales
    def extract_individual_emojis(text):
        return [c for c in text if c in emoji.EMOJI_DATA]

    # UDF para dividir lo emojis y entregar un arreglo
    extract_individual_emojis_udf = udf(extract_individual_emojis, ArrayType(StringType()))

    # Agregamos una columna "emojis" al DataFrame con emojis individuales (en un arreglo)
    df = df.withColumn("emojis", extract_individual_emojis_udf(df["content"]))
    
    # Desagrupamos los arreglos del campo emojis en filas separadas (emoji)
    df = df.select("emojis").withColumn("emoji", explode("emojis")).filter(col("emoji") != "")

    # Contamos la frecuencia de cada emoji en todo el DataFrame
    emoji_counts = df.groupBy("emoji").count()

    # Ordenamos los resultados por conteo en orden descendente
    emoji_counts = emoji_counts.orderBy(desc("count"))

    # Obtenemos los 10 emojis con mayor conteo
    top_10_emojis = emoji_counts.limit(10).collect()

    return top_10_emojis

#### Q3

In [66]:
def q3_time(file_path: str) -> List[Tuple[str, int]]:
    # Creamos la sesión de Spark y cargamos el archivo JSON en un DataFrame
    spark = SparkSession.builder.appName("TopMentionedUsers").getOrCreate()
    df_mention = spark.read.json(file_path)

    #Convertimos la columna date en
    df_mention = df_mention.withColumn("date", date_format(col("date"), "yyyy-MM-dd").cast("date"))

    #Unimos los tweets originales y los tweets en quotedTweet
    df_combined_mention = df_mention.select("mentionedusers.username").filter(col("mentionedUsers.username").isNotNull()).union(df_mention.select("quotedTweet.mentionedUsers.username").filter(col("quotedTweet.mentionedUsers.username").isNotNull()))

    #Descomponemos la columna "usernames" en filas individuales
    df = df_combined_mention.selectExpr("explode(username) as usernames")

    # Contamos el número de veces que aparece cada usuario
    user_counts = df.groupBy("usernames").count()

    #Ordenamos en orden descendente por conteo y seleccionar los 10 primeros usuarios
    top_10_users = user_counts.orderBy(col("count").desc()).limit(10)

    # Recopilamos los resultados en una lista de tuplas (usuario, conteo)
    result_list = [(row.usernames, row["count"]) for row in top_10_users.collect()]

    return result_list

In [67]:
# Modificar file_path:
file_path = "/home/david/Escritorio/farmers-protest-tweets-2021-2-4.json"
%time top_tweets_time = q1_time(file_path)
%time top_emojis_time = q2_time(file_path)
%time top_mentions_time = q3_time(file_path)

print('\n')
print(top_tweets_time)
print('\n')
print(top_emojis_time)
print('\n')
print(top_mentions_time)

23/09/26 15:59:47 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.
23/09/26 15:59:50 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


CPU times: user 20.2 ms, sys: 0 ns, total: 20.2 ms
Wall time: 3.12 s


23/09/26 15:59:52 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


CPU times: user 11.7 ms, sys: 264 µs, total: 12 ms
Wall time: 1.77 s


CPU times: user 6.7 ms, sys: 5.35 ms, total: 12 ms
Wall time: 1.85 s


[('2021-02-19', 'Preetm91'), ('2021-02-18', 'neetuanjle_nitu'), ('2021-02-13', 'MaanDee08215437'), ('2021-02-17', 'RaaJVinderkaur'), ('2021-02-16', 'jot__b'), ('2021-02-23', 'preetysaini321'), ('2021-02-15', 'jot__b'), ('2021-02-14', 'Gurpreetd86'), ('2021-02-20', 'MangalJ23056160'), ('2021-02-12', 'rebelpacifist')]


[Row(emoji='🙏', count=7286), Row(emoji='😂', count=3072), Row(emoji='🚜', count=2972), Row(emoji='✊', count=2411), Row(emoji='🌾', count=2363), Row(emoji='🏻', count=2080), Row(emoji='❤', count=1779), Row(emoji='🤣', count=1668), Row(emoji='🏽', count=1218), Row(emoji='👇', count=1108)]


[('narendramodi', 2623), ('Kisanektamorcha', 2045), ('RakeshTikaitBKU', 1848), ('PMOIndia', 1560), ('GretaThunberg', 1274), ('RahulGandhi', 1252), ('rihanna', 1142), ('DelhiPolice', 1134), ('RaviSinghKA', 1127), ('UNHumanRights', 1057)]


###      ENFOQUE OPTIMIZACIÓN DE MEMORIA

#### Q1

In [59]:
def q1_memory(file_path: str) -> List[Tuple[datetime.date, str]]:
    #Creamos la sesión de Spark y cargamos el archivo
    spark = SparkSession.builder.appName("TopDateUsers").getOrCreate()
    df = spark.read.json(file_path)

    #Convertimos la columna de fecha (para que tome solo lo necesario)
    df = df.withColumn("date", date_format(col("date"), "yyyy-MM-dd").cast("date"))

    # Unimos los tweets originales y los tweets en quotedTweet
    df_combined = df.select("date", "user.username").union(df.select("quotedTweet.date", "quotedTweet.user.username").filter(col("quotedTweet.date").isNotNull() & col("quotedTweet.user.username").isNotNull()))

    #df con las 10 fechas con más tweets
    #esta parte se podría parametrizar, para este caso es válido ya que sabemos a priori el tamaño a mostrar en la lista,
    #pero de ser un tamaño n no sería conveniente.
    
    #Para liberar memoria realizo primero la agregación (para contar la cantidad por fecha) y luego obtengo el top ordenandolos
    #En dos pasos separados
    agg_df = df_combined.groupBy("date").agg(count("*").alias("count"))
    
    # Ordenar los resultados
    top_dates = agg_df.orderBy(desc("count")).limit(10)
        
    #Obtenemos las 10 fechas con más tweets directamente como DataFrame
    top_dates_df = top_dates.select("date")

    # Obtenemos las 10 fechas como lista
    top_dates_list = [row.date for row in top_dates.collect()]

    # Filtrar el DataFrame original
    filtered_df = df_combined.filter(col("date").isin(top_dates_list))

    #Agrupamos el DataFrame por fecha y usuario y calcula el recuento de cada grupo
    grouped_df = filtered_df.groupBy("date", "username").agg(count("*").alias("count"))

    # Usamos una ventana para obtener el usuario que más se repite por fecha mediante rank
    window_spec = Window.partitionBy("date").orderBy(desc("count"))
    top_users_df = grouped_df.withColumn("rank", row_number().over(window_spec)).filter(col("rank") == 1)

    # Seleccionamos las columnas necesarias
    result_df = top_users_df.select("date", "username", "count")
    
    #Ordenamos el DataFrame por count en orden descendente
    result_df = result_df.orderBy(desc("count"))

    #Recopilamos los resultados en una lista de tuplas
    result = result_df.collect()

    #Conviertimos el resultado en una lista de tuplas
    result_list = [(row.date, row.username) for row in result]

    return result_list


#### Q2

In [60]:
def q2_memory(file_path: str) -> List[Tuple[str, int]]:
    # Creamos la sesión de Spark y cargar el archivo
    spark = SparkSession.builder.appName("TopEmojis").getOrCreate()
    df = spark.read.json(file_path)

    # Función para dividir los emojis en caracteres individuales
    def extract_individual_emojis(text):
        return [c for c in text if c in emoji.EMOJI_DATA]

    # UDF para dividir los emojis y entregar un arreglo
    extract_individual_emojis_udf = udf(extract_individual_emojis, ArrayType(StringType()))

    # Agregamos una columna "emojis" al DataFrame con emojis individuales (en un arreglo)
    df = df.withColumn("emojis", extract_individual_emojis_udf(df["content"]))
    
    # Desagrupamos los arreglos del campo emojis en filas separadas (emoji)
    df = df.select("emojis").withColumn("emoji", explode("emojis")).filter(col("emoji") != "")

    # Contamos la frecuencia de cada emoji en todo el DataFrame
    emoji_counts = df.groupBy("emoji").count()

    #Obtenemos los 10 emojis con mayor conteo (usando limit antes de ordenar)
    top_10_emojis = emoji_counts.orderBy(desc("count")).limit(10).collect()

    # Liberamos la memoria eliminando el DataFrame df
    df.unpersist()

    return top_10_emojis

#### Q3

In [61]:
def q3_memory(file_path: str) -> List[Tuple[str, int]]:
    # Creamos una sesión de Spark
    spark = SparkSession.builder.appName("TopMentionedUsers").getOrCreate()

    # Cargamos el archivo JSON en un DataFrame
    df_mention = spark.read.json(file_path)

    # Convertimos la columna date en el formato necesario
    df_mention = df_mention.withColumn("date", date_format(col("date"), "yyyy-MM-dd").cast("date"))

    # Unir los tweets originales y los tweets en quotedTweet
    df_combined_mention = df_mention.select(explode("mentionedUsers.username").alias("username"))

    # Contar el número de veces que aparece cada usuario
    user_counts = df_combined_mention.groupBy("username").count()

    # Ordenar en orden descendente por conteo y seleccionar los 10 primeros usuarios
    top_10_users = user_counts.orderBy(desc("count")).limit(10)

    # Recopilar los resultados en una lista de tuplas (usuario, conteo)
    result_list = [(row.username, row["count"]) for row in top_10_users.collect()]

    return result_list

In [62]:
# Modificar file_path:
file_path = "/home/david/Escritorio/farmers-protest-tweets-2021-2-4.json"
%time top_tweets_memory = q1_memory(file_path)
%time top_mentions_memory = q2_memory(file_path)
%time top_emojis_memory = q3_memory(file_path)

print('\n')
print(top_tweets_memory)
print('\n')
print(top_mentions_memory)
print('\n')
print(top_emojis_memory)

23/09/26 15:58:51 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.
23/09/26 15:58:54 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


CPU times: user 12.6 ms, sys: 8.86 ms, total: 21.4 ms
Wall time: 3.26 s


23/09/26 15:58:56 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


CPU times: user 11.7 ms, sys: 1.12 ms, total: 12.8 ms
Wall time: 1.94 s


CPU times: user 7.52 ms, sys: 0 ns, total: 7.52 ms
Wall time: 1.41 s


[('2021-02-19', 'Preetm91'), ('2021-02-18', 'neetuanjle_nitu'), ('2021-02-13', 'MaanDee08215437'), ('2021-02-17', 'RaaJVinderkaur'), ('2021-02-16', 'jot__b'), ('2021-02-23', 'preetysaini321'), ('2021-02-15', 'jot__b'), ('2021-02-14', 'Gurpreetd86'), ('2021-02-20', 'MangalJ23056160'), ('2021-02-12', 'rebelpacifist')]


[Row(emoji='🙏', count=7286), Row(emoji='😂', count=3072), Row(emoji='🚜', count=2972), Row(emoji='✊', count=2411), Row(emoji='🌾', count=2363), Row(emoji='🏻', count=2080), Row(emoji='❤', count=1779), Row(emoji='🤣', count=1668), Row(emoji='🏽', count=1218), Row(emoji='👇', count=1108)]


[('narendramodi', 2265), ('Kisanektamorcha', 1840), ('RakeshTikaitBKU', 1644), ('PMOIndia', 1427), ('RahulGandhi', 1146), ('GretaThunberg', 1048), ('RaviSinghKA', 1019), ('rihanna', 986), ('UNHumanRights', 962), ('meenaharris', 926)]
